<a href="https://colab.research.google.com/github/mostaf3-salama/My-Project-Graduation/blob/main/Clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import zipfile
import os
from google.colab import files
print("--- 1. Unzipping Individual Files ---")
# قائمة بأسماء الملفات المضغوطة كما تظهر في صورتك
zip_files_list = [
    'df_Customers.csv.zip',
    'df_OrderItems.csv.zip',
    'df_Orders.csv.zip',
    'df_Payments.csv.zip',
    'df_Products.csv.zip'
]
# حلقة تكرارية لفك ضغط كل ملف
for zip_file in zip_files_list:
    if os.path.exists(zip_file):
        try:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_ref.extractall('.')
            print(f"✅ Extracted: {zip_file}")
        except zipfile.BadZipFile:
            print(f"⚠️ Warning: {zip_file} is not a valid zip file.")
    else:
        print(f"❌ Error: File not found: {zip_file}")

print("--- 2. Loading Data ---")
# بعد فك الضغط، المتوقع أن نجد ملفات CSV بنفس الأسماء (بدون .zip)
try:
    orders = pd.read_csv('df_Orders.csv')
    items = pd.read_csv('df_OrderItems.csv')
    customers = pd.read_csv('df_Customers.csv')
    products = pd.read_csv('df_Products.csv')
    print("✅ All CSV Files loaded successfully.")
except FileNotFoundError as e:
    print("❌ Error: Could not find the CSV files after unzipping.")
    print("Make sure the zip files contained files named 'df_Orders.csv', etc.")
    print(f"System Error: {e}")
    raise

print("--- 3. Cleaning & Merging ---")
# توحيد عمود التاريخ
# نبحث عن العمود الذي يحتوي على كلمة date أو timestamp
date_col = [col for col in orders.columns if 'date' in col.lower() or 'timestamp' in col.lower()][0]
orders[date_col] = pd.to_datetime(orders[date_col])
# دمج الطلبات مع تفاصيل المنتجات
merged_df = pd.merge(orders, items, on='order_id', how='left')
# حساب إجمالي الفاتورة لكل طلب
order_totals = merged_df.groupby('order_id')['price'].sum().reset_index()
order_totals.rename(columns={'price': 'total_amount'}, inplace=True)
# إضافة الإجمالي لجدول الطلبات الرئيسي
orders = pd.merge(orders, order_totals, on='order_id', how='left')
orders['total_amount'].fillna(0, inplace=True)
print("✅ Data merged.")
print("--- 4. Calculating RFM ---")
# تحديد تاريخ اليوم الافتراضي
latest_date = orders[date_col].max() + dt.timedelta(days=1)
# تجميع البيانات
rfm = orders.groupby('customer_id').agg({
    date_col: lambda x: (latest_date - x.max()).days, # Recency
    'order_id': 'count',                              # Frequency
    'total_amount': 'sum'                             # Monetary
}).reset_index()
rfm.columns = ['customer_id', 'Recency', 'Frequency', 'Monetary']
# حساب الدرجات (Scoring)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])
# تحديد الشريحة (Segmentation)
def define_segment(row):
    r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])
    if r >= 4 and f >= 4 and m >= 4: return 'Champions'
    elif r >= 3 and f >= 3 and m >= 3: return 'Loyal'
    elif r <= 2 and f >= 4 and m >= 4: return 'At Risk'
    elif r >= 4 and f <= 2: return 'New Customers'
    elif r <= 2 and f <= 2: return 'Lost'
    else: return 'Potential'
rfm['Segment'] = rfm.apply(define_segment, axis=1)
# دمج مع بيانات العملاء
final_customers = pd.merge(customers, rfm, on='customer_id', how='left')
final_customers['Segment'].fillna('New', inplace=True)
print("✅ RFM Done.")
print("--- 5. Marketing Simulation ---")
def get_campaign(date_val):
    if pd.isnull(date_val): return 'Organic'
    m = date_val.month
    if m in [11, 12]: return 'Black Friday / Christmas'
    elif m in [1, 2]: return 'Winter Sale'
    elif m in [6, 7]: return 'Summer Offer'
    else: return 'Social Media Ads'
orders['Campaign_Name'] = orders[date_col].apply(get_campaign)
print("--- 6. Exporting & Downloading ---")
# حفظ الملفات النظيفة
final_customers.to_csv('Clean_Dim_Customers.csv', index=False)
orders.to_csv('Clean_Fact_Sales.csv', index=False)
products.to_csv('Clean_Dim_Products.csv', index=False)
# تحميلها لجهازك
print("Downloading files...")
files.download('Clean_Dim_Customers.csv')
files.download('Clean_Fact_Sales.csv')
files.download('Clean_Dim_Products.csv')
print("🎉 Process Complete!")

--- 1. Unzipping Individual Files ---
✅ Extracted: df_Customers.csv.zip
✅ Extracted: df_OrderItems.csv.zip
✅ Extracted: df_Orders.csv.zip
✅ Extracted: df_Payments.csv.zip
✅ Extracted: df_Products.csv.zip
--- 2. Loading Data ---
✅ All CSV Files loaded successfully.
--- 3. Cleaning & Merging ---


/tmp/ipython-input-4107289588.py:54: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  orders['total_amount'].fillna(0, inplace=True)


✅ Data merged.
--- 4. Calculating RFM ---


/tmp/ipython-input-4107289588.py:82: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_customers['Segment'].fillna('New', inplace=True)


✅ RFM Done.
--- 5. Marketing Simulation ---
--- 6. Exporting & Downloading ---


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 Process Complete!
